In [2]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import os
%load_ext dotenv
%dotenv

In [3]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

## Add images to storage account

In [ ]:
input_dir = "input"

Add the style image to storage.

In [ ]:
!azcopy \
    --source images/sample_renior.jpg \
    --destination https://{get_key(env_path, "STORAGE_ACCOUNT_NAME")}.blob.core.windows.net/{get_key(env_path, "STORAGE_CONTAINER_NAME")}/style_image.jpg \
    --dest-key {get_key(env_path, "STORAGE_ACCOUNT_KEY")} \
    --resume "." 

Add the images from the video to a test container, called "input_test", in your storage account.

In [ ]:
!azcopy \
    --source images/{get_key(env_path, "VIDEO_NAME")} \
    --destination https://{get_key(env_path, "STORAGE_ACCOUNT_NAME")}.blob.core.windows.net/{get_key(env_path, "STORAGE_CONTAINER_NAME")}/$input_dir \
    --dest-key {get_key(env_path, "STORAGE_ACCOUNT_KEY")} \
    --recursive \
    --resume "."

In [ ]:
set_key(env_path, "STORAGE_INPUT_DIR", input_dir)

## Add messages to queue

In [ ]:
!python queuer_app/add_images_to_queue.py \
    --input-dir $input_dir \
    --queue-limit 3

In [4]:
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query "countDetails.activeMessageCount"

823


## Test dequeueing

Test dequeuing from the service bus queue using the script `process_images_from_queue.py` inside `receiver_app`. Since we used the `--queue-limit` option when adding to the queue, we only have a few items to dequeue. However, we can also use the option `--dequeue-limit` with `process_images_from_queue.py` to dequeue a specified number of images.

In [ ]:
!python dequeuer_app/process_images_from_queue.py --input-dir $input_dir

Check that everything is dequeued.

In [ ]:
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query "countDetails.activeMessageCount"

---

## Conclusion

In [ ]:
!cat .env